In [ ]:
# path to mbatch-corrected one_cncr files:

In [ ]:
# paths to vae-corrected two_cncr files:

In [ ]:
# path to mbatch-corrected one_cncr files:

In [ ]:
# paths to vae-corrected two_cncr files: